# **Latent Dirichlet Allocation (LDA) for Topic Modeling Monkeypox Twitter Discourse**
By Minal Mishra

In [ ]:
!python -m pip install nltk
!python -m pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 6.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=e7c0fe41c4b39ef222cea5cb2e20333f872a7374a6af3c03e9fe0fca71d80dbc
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=eef9d2d5ec80f97a14b7922a27f2795507d1c7f296ce76cda29714fac0f6e21a
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built pyLDAvis sklearn


In [ ]:
# importing all modules
import numpy as np
import json
import glob

#gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from gensim.models import CoherenceModel

#spacy and nltk
import spacy
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

#pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


# Dataset Loading and Preprocessing

In [ ]:
import re
def clean_data(line):
  line = line.lower()   # convert all characters to lower case
  line = re.sub(r'[\'"]', " ", line) # to avoid removing contractions in english
  line = re.sub(r'\s+&amp[^A-Za-z0-9]', ' ', line) # removes &amp
  line = re.sub(r"@[A-Za-z0-9_]+","", line) # removes name mentions
  line = re.sub(r"#[A-Za-z0-9_]+","", line) # removes hashtags
  line = re.sub(r'http\S+', '', line) # removes all hyperlinks
  line = re.sub(r'\\n', ' ', line) # remove newline characters
  line = re.sub(r'[()!*?\[\]]', ' ', line) # remove punctuation marks
  line = re.sub(r"[^a-z0-9]"," ", line) # removes all characters except [a-z] and [0-9]
  line = line.split()
  line = [w for w in line if not w in stopwords]
  line = " ".join(word for word in line)
  line = line.strip()

  return line

fr = open("40k_all_tweets.txt")

tweets=[]
for line in fr:
  if not line.isspace():
    tweets.append(clean_data(line))
tweets=[str for str in tweets if str]

print(tweets)

data = tweets
fr.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
#lemmatisation
def lemmatization(texts, allowed_postags = ["NOUN", "ADJ", "VERB", "PROPN", "ADV"]):
  nlp = spacy.load("en_core_web_sm", disable=['parser','ner'])
  texts_out=[]

  for text in texts:
    doc = nlp(text)
    new_text=[]
    for token in doc:
      if token.pos_ in allowed_postags and not token.is_stop:
        new_text.append(token.lemma_)
    final = " ".join(new_text)
    texts_out.append(final)
  return texts_out

lemmatized_texts=lemmatization(data)


/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
#removing stopwords
from gensim.parsing.preprocessing import remove_stopwords

new_lemmatized_texts = []
for sentence in lemmatized_texts:
  new_lemmatized_texts.append(remove_stopwords(sentence))

lemmatized_texts = new_lemmatized_texts


# prerocessing text
def gen_words(texts):
  final = []
  for text in texts:
    new = gensim.utils.simple_preprocess(text, deacc= True, min_len= 3)
    final.append(new)
  return final

data_words=gen_words(lemmatized_texts)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



365411


In [ ]:
# removing custom stopwords

custom_stopwords = ['wtf','shit','nit','fuck','lol','oh']

new_data_words=[]
for sentence in data_words:
   new_sentence = [word for word in sentence if word not in custom_stopwords]
   new_data_words.append(new_sentence)

data_words = new_data_words

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Adding Bigrams to Vocabulary

from gensim.models import Phrases

bigrams = Phrases(data_words, min_count = 5)         

for sentence_id in range(len(data_words)):
  for token in bigrams[data_words[sentence_id]]:
    if '_' in token:
      data_words[sentence_id].append(token)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Creating TFIDF Embeddings

In [ ]:
# Creating the tfidf word embeddings for corpus of tweets

from gensim import corpora

#creating the vocabulary
dct = corpora.Dictionary(data_words)

#BoW embedding/model
BoW_corpus = [dct.doc2bow(doc) for doc in data_words]
print("Bow Corpus:", BoW_corpus)

#tfidf embedding/model
from gensim.models import TfidfModel

#tfidf_corpus = TfidfModel(BoW_corpus)
tfidf_corpus = TfidfModel(BoW_corpus, normalize=True, pivot=10, slope=0.25)

BoW_corpus = [i for i in BoW_corpus if i != []]
tfidf_corpus_final = tfidf_corpus[BoW_corpus]

dictionary: Dictionary(21446 unique tokens: ['change', 'defamation', 'fear', 'monkey', 'pox']...)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



'\n#creating dictionary\n# what can we do with the id2word dictionary object?\nid2word = corpora.Dictionary(data_words)\n\ncorpus=[]\n\nfor text in data_words:\n  new = id2word.doc2bow(text)\n  corpus.append(new)\n\nprint(corpus)\nprint(id2word[0][1])\nprint(id2word[0])\n'

# **Application of LDA Topic Model**

In [ ]:
# applying the LDA model
from gensim.models import LdaModel

num_topics=16
lda_model = LdaModel(tfidf_corpus_final, num_topics = num_topics, id2word=dct, chunksize=2000, passes=1, update_every=1, alpha='symmetric', eta=None, eval_every=10, iterations=50, random_state=42)


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

'\n#creating LDA gensim model\n\nlda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=15, random_state=100,update_every=1,chunksize=100,passes=10,alpha="auto")\n'

In [ ]:
print("Topic Word Distribution (word_probability*word pairs per topic):")
print(lda_model.get_topics())      # probabilities of each word in each topic, for all topics

#Creating topics (list of listt of words) for evaluation
topics_org=[]
for topicid in range(num_topics):
  topics_org.append([dct.id2token[word[0]] for word in lda_model.get_topic_terms(topicid, topn=10)])  # gives word_id, probability pairs in descending order of relevancy
print("\nTopics:\n",topics_org)

topics=[]
for words_per_topic in topics_org:
  topic=[]
  for word in words_per_topic:
    word_split=word.split('_')
    if len(word_split) == 1:
      topic.append(word)
    else:
      [topic.append(subword) for subword in word_split]
  topics.append(topic)

print("\nTopics:\n",topics)


Topic Word Distribution (word_probability*word pairs per topic):
[[1.5855048e-04 5.7221318e-06 3.2630994e-03 ... 7.4190852e-05
  7.4190837e-05 5.8328803e-04]
 [5.8982574e-04 4.4531066e-06 4.4531116e-06 ... 6.3996406e-05
  6.3996406e-05 4.4531298e-06]
 [5.0007179e-06 5.0019858e-06 5.5585720e-06 ... 5.0007134e-06
  5.0007134e-06 5.0007388e-06]
 ...
 [2.1823116e-04 4.8453835e-06 4.8385928e-06 ... 1.2029150e-05
  1.2029150e-05 4.8386160e-06]
 [4.8233583e-06 4.8233551e-06 1.4373721e-03 ... 4.8233564e-06
  4.8233564e-06 4.8233801e-06]
 [6.2719378e-06 6.2719278e-06 5.1869988e-04 ... 6.2719323e-06
  6.2719323e-06 6.2719737e-06]]

Topics:
 [['alert', 'high', 'declare_high', 'fauci', 'monkeypox', 'lab', 'source', 'funny', 'news', 'update'], ['vaccine', 'smallpox', 'smallpox_vaccine', 'month', 'pox', 'monkeypox', 'pride', 'monkey', 'year', 'expect'], ['pox', 'monkey', 'skin', 'covid', 'believe', 'worry', 'human', 'spread', 'season', 'maybe'], ['covid', 'pox', 'monkey', 'hope', 'guess', 'good', 't

'\n#Rough for Topic Representation in LDA\nprint(lda_model.get_topics())       #gives term-topic matrix -> the probability for each word in each topic\n\nfor i in lda_model.get_topics():   \n  print(len(i)) \n\nfor topicid in range(num_topics):\n  print(lda_model.get_topic_terms(topicid, topn=10))    # gives word_id, probability pairs in descending order of relevancy\nprint("\n\n Topic Word Distribution:")\nprint(lda_model.show_topics(num_topics=10, num_words=10, log=False, formatted=True))  # gives (topic_no, probability*word_name) for each topic; probility = probability of the word belonging to the topic\n\nprint("\n\n TEST:")\nprint(lda_model.get_topics())      # probabilities of each word in each topic, for all topics\n\n#print([(i,j) for (i,j) in dct.items()])\n\n\n\n\ntopics_org=[]\nfor topicid in range(num_topics):\n  topics_org.append([dct.id2token[word[0]] for word in lda_model.get_topic_terms(topicid, topn=10)])  # gives word_id, probability pairs in descending order of relev

# **Evaluation**

In [ ]:
# Evaluating the topics - Topic Coherence
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel


#creating BoW corpus for evaluation
nlp_eval = spacy.load("en_core_web_sm", disable=['parser','ner'])

def clean_data_for_eval(line):
  line = line.lower()   # convert all characters to lower case
  line = re.sub(r'[\'"]', " ", line) # to avoid removing contractions in english
  line = re.sub(r'\s+&amp[^A-Za-z0-9]', ' ', line) # removes  &amp
  line = re.sub(r"@[A-Za-z0-9_]+","", line) # removes name mentions
  line = re.sub(r"#[A-Za-z0-9_]+","", line)  # remove hashtags
  line = re.sub(r'http\S+', '', line) # removes hyperlinks
  line = re.sub(r'\\n', ' ', line) # removes newline characters
  line = re.sub(r'[()!*?\[\]]', ' ', line) # removes punctuation
  line = re.sub(r"[^a-z0-9]"," ", line) # removes all characters except [a-z] and [0-9]
  line = line.split()
  line = [w for w in line if not w in stopwords]
  line = " ".join(word for word in line)
  line = line.strip()
  doc = nlp_eval(line)
  new_line=[]

  for token in doc:
    new_line.append(token.lemma_)
  line = " ".join(new_line)

  return line

fr_eval = open("/content/40k_all_tweets.txt")

tweets=[]
for line in fr_eval:
  if not line.isspace():
    tweets.append(clean_data_for_eval(line))
tweets=[str for str in tweets if str]
print(tweets)

data = tweets

texts = [[word for word in str(document).split()] for document in tweets]
print(texts)
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

fr_eval.close()

coherence_model = CoherenceModel(topics=topics, texts=texts, corpus=corpus, dictionary=id2word, coherence='c_v')
coherence = coherence_model.get_coherence()
print("\nCoherence Score:",coherence)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Coherence Score: 0.4730892374141012


# **Visualization of Topics**

In [ ]:
# Vizualization of LDA Model

pyLDAvis.enable_notebook()

viz = pyLDAvis.gensim_models.prepare(lda_model, tfidf_corpus_final, dct)

viz


/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
9     -0.021789 -0.099975       1        1  8.077782
11     0.086037  0.113894       2        1  7.190639
6      0.291861  0.042339       3        1  6.786909
1     -0.046379 -0.055115       4        1  6.725477
8     -0.019769 -0.113685       5        1  6.722762
7     -0.115080  0.125230       6        1  6.694038
3     -0.122491  0.052027       7        1  6.354854
14    -0.103179  0.069834       8        1  6.273067
13    -0.021568 -0.094784       9        1  6.132839
12    -0.105272  0.060277      10        1  6.067858
5     -0.003370 -0.121822      11        1  5.974989
2     -0.088517  0.068576      12        1  5.905210
4      0.026888 -0.159744      13        1  5.724115
0      0.072112 -0.075255      14        1  5.369390
10     0.208186  0.104224      15        1  5.074929
15    -0.037670  0.083980      16        1  4.925142, topic_info=             Term        Freq        Total Category  logprob  loglift
138     emergency  711.000000   711.000000  Default  30.0000  30.0000
211        global  619.000000   619.000000  Default  29.0000  29.0000
131        health  864.000000   864.000000  Default  28.0000  28.0000
137       declare  593.000000   593.000000  Default  27.0000  27.0000
148      moneypox  281.000000   281.000000  Default  26.0000  26.0000
...           ...         ...          ...      ...      ...      ...
35      monkeypox   59.375240  1544.461997  Topic16  -5.0369  -0.2477
24        vaccine   51.429230   890.369954  Topic16  -5.1805   0.1594
3          monkey   54.868395  1509.063733  Topic16  -5.1158  -0.3035
4             pox   52.957006  1530.095397  Topic16  -5.1513  -0.3528
1013  declaration   34.001104    48.394403  Topic16  -5.5944   2.6578

[1011 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
3288      6  0.141836  abortion
3288      8  0.851019  abortion
230      14  0.972976    accept
392       1  0.086232    accord
392       3  0.181088    accord
...     ...       ...       ...
3139      3  0.986043      yike
418       5  0.869044      york
418       8  0.120462      york
986      15  0.970948    yorker
3323      6  0.970798   youtube

[2732 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 12, 7, 2, 9, 8, 4, 15, 14, 13, 6, 3, 5, 1, 11, 16])